# Up and Running with TensorFlow

*TensorFlow* is a powerful open-source software library for numerical computation, particularly well suited and fine-tuned for large-scale machine learning. Its basic principle is simple; first define in Python a graph of computations to perform, and then TensorFlow takes the graph and runs it efficiently using optimized C++.

Most importantly, it's possible to break the graph into several chunks and run them in parallel across multiple CPUs/GPUs. TensorFlow also supports distributed computing, so you can train colossal neural networks on humongous training sets in a reasonable amount of time by splitting the computations across hundreds of servers (see chapter 12). TensorFlow can train a network with millions of parameters on a training set composed of billions of instances with millions of features each. This shouldn't be a surprise, since TensorFlow was developed by Google's Brain team and powers many things like Google Cloud Speech, Google Photos, and Google Search

Here's a table of open source Deep Learning libraries available (not exhaustive):

Library | API | Platforms | Started by | Year
--- | --- | --- | --- | ---
Caffe | Python, C++, Matlab | Linux, macOS, Windows | y, Jia, UC Berkeley | 2013
Deeplearning4j | Java, Scala, Clojure | Linux, macOS, Windows, Android | A. Gibson, J. Patterson | 2014
H20 | Python, R | Linux, macOS, Windows | H20.ai | 2014
MXNet | Python, C++, others | Linux, macOS, Windows, iOS, Android | DMLC | 2015
TensorFlow | Python, C++ | Linux, macOS, Windows, iOS, Android | Google | 2015
Theano | Python | Linux, macOS, iOS | University of Montreal | 2010
Torch | C++, Lua | Linux, macOS, iOS, Android | R. Collobert, K. Kavukcuoglu, C. Farabet | 2002

## Creating Your First Graph and Running It in a Session

In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
import numpy as np
import tensorflow as tf

x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

In [11]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
result

42

In [12]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
    
result

42

In [13]:
# Alternatively, you can create an interactive session
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


A TensorFlow program is typically split into two parts: the first part builds a computation graph (this is called the *construction phase*), and the second part runs it (this is called the *execution phase*). The construction phase typically builds a computation graph represeting the ML model and the computations required to train it. The execution phase generally runs a loop that evaluates a training step repeatedly (for example, one step per mmini-batch), gradually improving the model parameters. WE well go through an example shortly.

## Managing Graphs

Any node you create is automatically added to the default graph:

In [14]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

This is fine in most cases, but sometimes you may want to manage multiple independent graphs. This can be done by creating a new `Graph` and temporariliy making it the default graph inside of a `with` block, like so:

In [15]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
    
x2.graph is graph

True

In [16]:
x2.graph is tf.get_default_graph()

False

*Note: in Jupyter (and in a Python shell) it's common to run the same commands repeatedly when experimenting. As a result, you may end up with a default graph containing multiple duplicate nodes. One solution is to restart the Jupyter kernel/Python shell, but a more convenient solution is to just reset the default graph by running `tf.reset_default_graph()`*

## Lifecycle of a Node Value

When you evaluate a node, TensorFlow automatically determines the set of nodes that it depends on and it evalutes those nodes first. For example, consider the following:

In [17]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval()) # 10
    print(z.eval()) # 15

10
15


First, this code defines a simple graph. Then it starts a session and runs the graph to evaluate `y`; TensorFlow will detect that `y` depends on `x` and that `x` depends on `w`, so it first will evaluate `w`, then `x`, then `y`, then returns the value of `y`. Finally, it will execute `z`. Once again, it detects that it needs `w` and `x`, but it *__will not__* reuse the result of the previous evaluation. In short, the above code evalutes the values of `x` and `w` twice.

All node values are dropped between graph runs, except variable values, which are maintained by the session across graph runs. A variable starts its life when its initializer (constructor) is run, and ends when the session is closed.

If you want to evalute `y` and `z` efficiently without evaluating `w` and `x` twice (like the previous code) you must ask Tensorflow to evaluate both `y` and `z` in just one graph run, as seen below:

In [18]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


*Note: in single-process TensorFlow, multiple sessions don't share any state, even if they reuse the same graph. In distributed TensorFlow (see chapter 12), variable state is stored on the servers, not in the sessions, so multiple sessions can share the same variables.

## Linear Regression with TensorFlow

TensorFlow operations (known as *ops* for short), can take any number of inputs and produce any number of outputs. For example, addition and multiplication both take two inputs and produce one output. Constants and variables are known as *source ops* (ops that take no input). The inputs and outputs are multidimensional arrays known as *tensors*. Just like Numpy arrays, tensors have a type and shape (in fact, the Python API tensors are just Numpy ndarrays. They typically contain floats, but you can use them to carry strings as well (arbitrary byte arrays)).

In our examples, the tensors have just contained a single scalar value, but you can (obviously) perform computations on arrays of any shape.

The following code performs Linear Regression on the California housing data from earlier by manipulating 2D arrays. It starts by fetching the data, then adding an extra bias input feature to all training instances using Numpy, then it creates two TensorFlow constant nodes `X` and `y`, to hold this data and the targets, and it uses some matrix operations provided by TensorFlow to define `theta`. You may recognize that `theta` is the Normal Equation ($\hat{\theta} = (\textbf{X}^T\cdot\textbf{X})^{-1}\cdot\textbf{X}^T\cdot y$; see chapter 4)Finally, the code creates a session and evaluates `theta`:

In [19]:
from sklearn.datasets import fetch_california_housing
from os.path import expanduser

housing = fetch_california_housing(data_home=expanduser('~/Coding Stuff/Python/handson-ml/datasets'))
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)

In [20]:
# Numpy version of what we just did
X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

theta_numpy

array([[-3.69419202e+01],
       [ 4.36693293e-01],
       [ 9.43577803e-03],
       [-1.07322041e-01],
       [ 6.45065694e-01],
       [-3.97638942e-06],
       [-3.78654266e-03],
       [-4.21314378e-01],
       [-4.34513755e-01]])

In [23]:
# scikit learn version
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1, 1))

print(np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


/usr/local/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


The main benefit of this code vs using Numpy to compute the Normal Equation is that TensorFlow will use your GPU if you have one (providing you installed TensorFlow with GPU support (see chapter 12)).

## Implementing Gradient Descent